In [2]:
# pip install nltk
# pip install transformers
# pip install lime

import sys
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from lime import lime_text
from lime.lime_text import LimeTextExplainer

import pickle, re, spacy
import xgboost as xgb
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import spacy
from flask import Flask, request, jsonify
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

In [8]:
# preprocessing functions
# # Preprocessing functions
# add space before punctuations

app = Flask(__name__)
explainer = LimeTextExplainer(class_names=['female', 'male'])
stemmer = PorterStemmer()
nlp = spacy.load("en_core_web_sm")

# remove gendered pronounds, names, stop words, and apply stemming
def removeUnnecessaryWords(text):
    doc = nlp(text)

    result = " ".join([
        "" if (
            token.pos_ == "PRON" and token.lemma_ not in ["I", "you"]
        ) or (
            token.ent_type_ == "PERSON" or token.text.lower() in ["woman", "women", "man", "men", "he", "she", "him", "her"]
        ) or (
            token.text.lower() in STOP_WORDS
        ) else stemmer.stem(token.lemma_) for token in doc])

    return result.strip()

SVM using BUG dataset

In [14]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

# preprocess data
print("PREPROCESSING DATA")
train['sentence_text'] = train['sentence_text'].apply(removeUnnecessaryWords)
test['sentence_text'] = test['sentence_text'].apply(removeUnnecessaryWords)
test.dropna(subset=['sentence_text'], inplace=True)
print("TRAIN", train['sentence_text'])
print("TEST", test['sentence_text'])

vectorizer = TfidfVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = LinearSVC()
model.fit(xTrain, yTrain)

model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
model.fit(xTrain, yTrain)

with open('../savedModels/svmModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

PREPROCESSING DATA
TRAIN 6224     writer cast      book , write   invit   munici...
17662    driver   accomplic  stop      investig   outbu...
13225    ( septemb 10 , 1921 – decemb 7 , 1941 )    nav...
12936    accord   version ,   sink   loss   hand   squa...
2457     leadership ,  student popul soon outgrow  lind...
                               ...                        
16850         film revolv   politician    (   )   famili .
6265     mountain cave   far   academi hold  massiv arm...
5390                teacher  author say    ` `     known .
860      articl draw   evid base   experi gain   author...
15795    bold  deserv editor    afraid  speak     feel ...
Name: sentence_text, Length: 10190, dtype: object
TEST 19059    artist accompani      sketch   journey ,   con...
4036     psychologist lose  profession licens  have  se...
14564    addit ,  real - life peopl  appear   ,   toriy...
14469    polic team (  sera   member ) investig  myster...
5652     autobiographi go  great de

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [15]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

# f1 score
f1 = f1_score(yTest, pred, average='weighted')
print("F1 Score:", f1)

Accuracy: 0.9578454332552693
Confusion matrix:
 [[1230   65]
 [  43 1224]]
F1 Score: 0.9578462809953023


In [16]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
    
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  0.6464656516721605
Female:  0.3535343483278395
Predicted:  male
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('the', 0.3245240488426354), ('to', 0.1500749383242496), ('Mia', 0.14928207822525708), ('The', 0.0314616445578306), ('urban', 0.02753740138313761), ('develop', 0.02085991378485671)]
Feminine words:  [('student', -0.27682777281955645), ('and', -0.2249581665682038), ('impact', -0.04043574368921398), ('passion', -0.03521335395041176), ('journey', -0.022893274737822635), ('in', -0.0210283723998051), ('planet', -0.02095349580658438), ('spread', -0.020496815569517487), ('scholarship', -0.020310680618775143), ('And', -0.019048662751110054), ('Rodriguez', -0.018265758911088528), ('make', -0.017183446428673435), ('even', -0.016319957049548642), ('vibrant', -0.01591205692170526)]

2 :
Male:  0.5466342810161908
Female:  0.4533657189838092
Predicted:  male
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('the', 0.4305442278074855), ('to', 0.1720528425411286), ('urban', 0.03916824806155053), ('The', 0.03783387718323002), ('at', 0.02091336348155185), ('new', 0.019104827333655698)]
Feminine words:  [('student', -0.3267271472660511), ('and', -0.23876296678781708), ('passion', -0.0471117981112895), ('impact', -0.039793872985952855), ('junior', -0.02642019480927222), ('in', -0.024782050375817797), ('scholarship', -0.024358807390183892), ('make', -0.020797298951132515), ('vibrant', -0.020031402406068998), ('even', -0.018066104877293744), ('planet', -0.01763955410603873), ('drew', -0.015457585795938204), ('journey', -0.015261071724112753), ('day', -0.014547240377352562)]

3 :
Male:  0.07736458680188667
Female:  0.9226354131981134
Predicted:  female
Actual: Male


...

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 8:
Masculine words:  [('the', 0.35050598407358363), ('to', 0.291377122141792), ('local', 0.040164704945725466), ('The', 0.030808956008295762), ('at', 0.024803437900735144), ('To', 0.02170306363864539)]
Feminine words:  [('and', -0.231146312728053), ('librarian', -0.22005113042125382), ('journalist', -0.16544481380035136), ('student', -0.14760345246381237), ('tale', -0.03697400305550957), ('in', -0.036422145505573225), ('Mrs', -0.03480835291719741), ('children', -0.03394998580421276), ('had', -0.030852432583014924), ('share', -0.029255682702786164), ('journey', -0.028994293135736586), ('joy', -0.026672162830942293), ('life', -0.02663108312832772), ('character', -0.02562096373139591)]

9 :
Male:  0.8967959226281229
Female:  0.10320407737187709
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('the', 0.24482391057692232), ('to', 0.1867912465568743), ('The', 0.06057183811307637), ('design', 0.033923983703660905), ('space', 0.03300741679778967), ('local', 0.02808385662894992), ('One', 0.02085824772965293), ('3D', 0.01782306996098586), ('system', 0.01582793400629893)]
Feminine words:  [('and', -0.1628639256986939), ('workshop', -0.0909180465343992), ('him', -0.04619972853906252), ('life', -0.02712257594297792), ('address', -0.025601833576315392), ('in', -0.022182693565790744), ('town', -0.01940972174341763), ('passion', -0.018143069022477743), ('pride', -0.01810226682155591), ('impact', -0.0156269365354828), ('Turner', -0.01395495732902042)]

10 :
Male:  0.9150312493722752
Female:  0.0849687506277248
Predicted:  male
Actual: Male
Top words for text response 10:
Masculine words:  [('the', 0.2135547583329217), ('to', 0.17301900462827893), ('The', 0.054921677006407515), ('local', 0.02824509267721187), ('space', 0.02574846744988238

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Random Forest Using BUG Dataset

In [17]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

# preprocess data
print("PREPROCESSING DATA")
train['sentence_text'] = train['sentence_text'].apply(removeUnnecessaryWords)
test['sentence_text'] = test['sentence_text'].apply(removeUnnecessaryWords)
test.dropna(subset=['sentence_text'], inplace=True)
print("TRAIN", train['sentence_text'])
print("TEST", test['sentence_text'])

# vectorize data
vectorizer = TfidfVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(xTrain, yTrain)

with open('../savedModels/randomForestModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

PREPROCESSING DATA
TRAIN 6224     writer cast      book , write   invit   munici...
17662    driver   accomplic  stop      investig   outbu...
13225    ( septemb 10 , 1921 – decemb 7 , 1941 )    nav...
12936    accord   version ,   sink   loss   hand   squa...
2457     leadership ,  student popul soon outgrow  lind...
                               ...                        
16850         film revolv   politician    (   )   famili .
6265     mountain cave   far   academi hold  massiv arm...
5390                teacher  author say    ` `     known .
860      articl draw   evid base   experi gain   author...
15795    bold  deserv editor    afraid  speak     feel ...
Name: sentence_text, Length: 10190, dtype: object
TEST 19059    artist accompani      sketch   journey ,   con...
4036     psychologist lose  profession licens  have  se...
14564    addit ,  real - life peopl  appear   ,   toriy...
14469    polic team (  sera   member ) investig  myster...
5652     autobiographi go  great de

In [18]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

# f1 score
f1 = f1_score(yTest, pred, average='weighted')
print("F1 score:", f1)

Accuracy: 0.9473067915690867
Confusion matrix:
 [[1239   56]
 [  79 1188]]
F1 score: 0.9472973712300611


In [19]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
        
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  0.84
Female:  0.16
Predicted:  male
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('the', 0.15959995415265635), ('Mia', 0.09505205818172364), ('and', 0.046689178953480896), ('to', 0.04138255313414777), ('One', 0.015006603356208532), ('The', 0.01351485778696744), ('legacy', 0.011006058104588984), ('for', 0.010739115410519879), ('as', 0.009542184739309038), ('And', 0.00866249182785871), ('people', 0.007856119714350566)]
Feminine words:  [('student', -0.1350581808820676), ('knowledge', -0.012922329605577864), ('inspiring', -0.008430976373977632), ('Green', -0.008216184937689065), ('think', -0.008057545084840984), ('garden', -0.007975083066287579), ('friendly', -0.007859929726169772), ('passion', -0.007751351896303874), ('meaningful', -0.005702902841401049)]

2 :
Male:  0.82
Female:  0.18
Predicted:  male
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('the', 0.2883997845125856), ('and', 0.065475148462763), ('to', 0.05114845220581566), ('The', 0.02461978893684196), ('did', 0.012350053567124654), ('As', 0.010904163150242797), ('as', 0.010467452258403915), ('Inspired', 0.01016982364123111), ('think', 0.009821837878320444), ('from', 0.009313172756744741), ('even', 0.009260929043526824)]
Feminine words:  [('student', -0.19937692899927506), ('green', -0.018072413364095617), ('Green', -0.010988184523869921), ('arrived', -0.008324346278870222), ('continued', -0.007237023385020885), ('studies', -0.007176745288707999), ('passion', -0.006379650371815221), ('judges', -0.0061499779945677355), ('vibrant', -0.00510168038293813)]

3 :
Male:  0.13
Female:  0.87
Predicted:  female
Actual: Male


...

Top words for text response 8:
Masculine words:  [('the', 0.10839851750780653), ('to', 0.08687258740731982), ('and', 0.025878699332397725), ('had', 0.025021676907314817), ('in', 0.022141777524147442), ('upon', 0.012311146282419342), ('The', 0.011582353923841923), ('time', 0.010404179464732497), ('couldn', 0.008089010697328335), ('instilled', 0.00798872950891273), ('into', 0.007876215161796641), ('aspiring', 0.007586739316628404)]
Feminine words:  [('librarian', -0.041206810973900374), ('share', -0.031814985498778016), ('life', -0.020262290163929737), ('student', -0.01670049903752314), ('human', -0.009985536795907819), ('storytelling', -0.009806586077830366), ('journalist', -0.009372311363098691), ('keen', -0.00807811177388698)]

9 :
Male:  0.84
Female:  0.16
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('the', 0.040894200230665444), ('to', 0.03064106180524801), ('The', 0.018377604090944515), ('design', 0.007302807654373571), ('breathe', 0.0048116572381474276), ('new', 0.004088987468860399), ('robotics', 0.003864792488193926), ('work', 0.0035215580268275547), ('system', 0.003385038064932688), ('metal', 0.0032192662342991887), ('minded', 0.0028976702289101896)]
Feminine words:  [('life', -0.029544329812823008), ('him', -0.015547083319143173), ('in', -0.006360392370413028), ('One', -0.005207315785969437), ('workshop', -0.0050441353191045925), ('their', -0.003687229760747852), ('for', -0.003603006336225008), ('just', -0.0029086117037752024), ('also', -0.002898709836780572)]

10 :
Male:  0.84
Female:  0.16
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 10:
Masculine words:  [('the', 0.0283732583024404), ('to', 0.019401410532901724), ('The', 0.016450522178666355), ('design', 0.005833872789405493), ('not', 0.004023176922154089), ('town', 0.003814996112481439), ('showcasing', 0.0029960842656070407)]
Feminine words:  [('life', -0.028451386625227434), ('and', -0.008910311291129588), ('in', -0.0036472815061764988), ('One', -0.0030276735106193396), ('workshop', -0.0021892532082810195), ('passion', -0.0019922638196789937), ('tools', -0.0010665960117800146), ('attention', -0.001029889388310798), ('innovation', -0.0010138166625127689), ('Inspired', -0.0009667853417630723), ('journey', -0.0009254983218353691), ('night', -0.0008857923526415758), ('world', -0.0008776342556447688)]



XGBoost

In [21]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
dataset.replace("male", 1, inplace=True)
dataset.replace("female", 0, inplace=True)
print(dataset.head())
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

# preprocess data
print("PREPROCESSING DATA")
train['sentence_text'] = train['sentence_text'].apply(removeUnnecessaryWords)
test['sentence_text'] = test['sentence_text'].apply(removeUnnecessaryWords)
test.dropna(subset=['sentence_text'], inplace=True)
print("TRAIN", train['sentence_text'])
print("TEST", test['sentence_text'])

vectorizer = TfidfVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
model.fit(xTrain, yTrain, eval_set=[(xTest, yTest)])

with open('../savedModels/XGBoostModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

   Unnamed: 0                                      sentence_text  \
0           0  Notice of contact depends on the time the infe...   
1           1  Hepaprotective agent glycyrrhizinate was added...   
2           2  So yes , that was humbling . " Addressing the ...   
3           3  Following previous works based on classical pr...   
4           4  The second aspect is the agent neighborhood ! ...   

                                              tokens profession    g  \
0  ['Notice', 'of', 'contact', 'depends', 'on', '...     person  her   
1  ['Hepaprotective', 'agent', 'glycyrrhizinate',...      agent  her   
2  ['So', 'yes', ',', 'that', 'was', 'humbling', ...  assistant  her   
3  ['Following', 'previous', 'works', 'based', 'o...      agent  her   
4  ['The', 'second', 'aspect', 'is', 'the', 'agen...      agent  her   

   profession_first_index  g_first_index  predicted gender  stereotype  \
0                       9             13                 0           1   
1         

In [22]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

# f1 score
f1 = f1_score(yTest, pred, average='weighted')
print("F1 score:", f1)

Accuracy: 0.9547228727556596
Confusion matrix:
 [[1232   63]
 [  53 1214]]
F1 score: 0.9547241144495836


In [23]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]

vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['Female', 'Male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
        
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")

1 :
Male:  0.82934874
Female:  0.17065126
Predicted:  1
Actual: Female
Top words for text response 1:
Masculine words:  [('became', 2.0297530618130244e-08), ('thrive', 1.960943650325435e-08), ('into', 1.7608570214956776e-08), ('connection', 1.5974730023093108e-08), ('contribute', 1.42261150469729e-08), ('vibrant', 1.3451314243604581e-08)]
Feminine words:  [('student', -1.5479547183911457e-06), ('new', -3.589134778051903e-07), ('and', -8.104390180895721e-08), ('Mia', -7.410626840331753e-08), ('the', -6.907755277512949e-08), ('to', -5.911859356432405e-08), ('The', -1.9596992831923637e-08), ('dedication', -1.649275295338384e-08), ('further', -1.588054399121798e-08), ('drew', -1.4876979830497029e-08), ('hub', -1.429448361240899e-08), ('peers', -1.3408049190694667e-08), ('science', -1.1594036740183868e-08), ('world', -9.819476864196837e-09)]

2 :
Male:  0.82934874
Female:  0.17065126
Predicted:  1
Actual: Female
Top words for text response 2:
Masculine words:  [('eco', 2.9701035798869182e-0

BERT (very slow run on Google Colab or high performance machine)

In [ ]:
# load data
dataset = pd.read_csv('gold_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

# preprocess data
print("PREPROCESSING DATA")
train['sentence_text'] = train['sentence_text'].apply(add_space_before).apply(removeUnnecessaryWords)
test['sentence_text'] = test['sentence_text'].apply(add_space_before).apply(removeUnnecessaryWords)
print("TRAIN", train['sentence_text'])
print("TEST", test['sentence_text'])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

xTrain = tokenizer(list(train['sentence_text']), padding=True, truncation=True, return_tensors='pt')
xTest = tokenizer(list(test['sentence_text']), padding=True, truncation=True, return_tensors='pt')
xTrain = xTrain['input_ids'].numpy(), xTrain['attention_mask'].numpy(), xTrain['token_type_ids'].numpy()
xTest = xTest['input_ids'].numpy(), xTest['attention_mask'].numpy(), xTest['token_type_ids'].numpy()


yTrain = train['predicted gender']
yTest = test['predicted gender']

FileNotFoundError: [Errno 2] No such file or directory: 'gold_BUG.csv'

In [ ]:
# train model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

yTrain = pd.get_dummies(yTrain)
yTest = pd.get_dummies(yTest)

model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=3)

model.save_pretrained('/content/drive/My Drive/mlModels/bertModel')
tokenizer.save_pretrained('/content/drive/My Drive/mlModels/bertTokenizer')

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('/content/drive/My Drive/mlModels/bertModel')
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/mlModels/bertTokenizer')

# accuracy
pred = model.predict(xTest).logits
pred = np.argmax(pred, axis=1)
yTest = np.argmax(yTest, axis=1)

pred = pred.tolist()
accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

In [ ]:
# Load the BERT model and tokenizer
model = TFBertForSequenceClassification.from_pretrained('/content/drive/My Drive/mlModels/bertModel')
tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/mlModels/bertTokenizer')

# List of texts to analyze
text = [
    """
    Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
    """,
    """
    Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
    """,
    """
    Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
    """,
    """
    Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
    """,
    """
    Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
    """,
]

# Define the actual labels (0 for Female, 1 for Male)
y_actual = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # Example labels, replace with actual labels

# Define a function for prediction
def predict_proba(texts):
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")
    tokenized_texts = tokenized_texts['input_ids'].numpy(), tokenized_texts['attention_mask'].numpy(), tokenized_texts['token_type_ids'].numpy()
    logits = model.predict(tokenized_texts).logits
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return probabilities

# Use LIME to explain predictions
explainer = LimeTextExplainer(class_names=["Female", "Male"])
pred_proba = predict_proba(text[0])
print("PRED", pred_proba)
# Define a lambda function to predict probabilities for a single text instance
predict_proba_fn = lambda x: predict_proba(x)

# Explain the prediction using LIME
explanation = explainer.explain_instance(text[0], predict_proba_fn, num_features=10)
explanation.show_in_notebook(text=True)

Word Embeddings with training model through dataset
Requirements: python -m spacy download en_core_web_md

In [24]:
# load spacy pre trained word embeddings
nlp = spacy.load("en_core_web_md") 

def text_to_vectors(text):
    # doc = nlp(text)
    # return doc.vector
    return nlp(text).vector

In [30]:
# load data
dataset = pd.read_csv('../datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# preprocess data
print("PREPROCESSING DATA")
train['sentence_text'] = train['sentence_text'].apply(removeUnnecessaryWords)
test['sentence_text'] = test['sentence_text'].apply(removeUnnecessaryWords)
test.dropna(subset=['sentence_text'], inplace=True)
print("TRAIN", train['sentence_text'])
print("TEST", test['sentence_text'])

# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

# convert xTrain and xTest to vectors based on word embeddings
xTrain = np.array([text_to_vectors(text) for text in train['sentence_text']])
xTest = np.array([text_to_vectors(text) for text in test['sentence_text']])
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(xTrain, yTrain)

# save model
with open('../savedModels/randomForestModelSpacyEmbeddings.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

PREPROCESSING DATA


In [ ]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

# f1 score
f1 = f1_score(yTest, pred, average='weighted')
print("F1 score:", f1)

Accuracy: 0.7423887587822015
Confusion matrix:
 [[961 334]
 [326 941]]
F1 score: 0.742395038686257


In [ ]:
with open('../savedModels/randomForestModelSpacyEmbeddings.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
Once upon a time, in the bustling city of Academia, there lived an ambitious scientist named Alex. He was a dedicated soul with a passion to unravel the mysteries of science. Alex led an insatiable curiosity that led him to the hallowed halls of the prestigious Kepler University. From a humble background, Alex worked tirelessly to secure a scholarship that would pave the way to his academic journey. The moment he stepped onto the university grounds, he felt a surge of determination to make the most of the golden opportunity. Alex's days were filled with challenging lectures, fascinating experiments, countless hours in the library. His favorite haunt was the physics lab, where he immersed himself in the world of particles, equations, the boundless wonders of the universe. The spark in his eyes ignited with each breakthrough, no matter the size. Outside the realm of academia, Alex was known to be kind to lend a helping hand. Tutoring a struggling classmate or participating in community service projects, he believed in the power of knowledge to uplift not just himself but those around him. As the semesters passed, Alex's reputation as a brilliant, compassionate scientist grew. Professors admired his commitment to learning; fellow scientists sought his guidance. Alex remained humble, always acknowledging the contributions of others, fostering a sense of camaraderie.
""",
"""
bustl town  summitvil ,  live  colleg  name   . luca   mechan engin major  summitvil tech , know   passion  build thing   ground  .  colleg life   dynam mix  late -night design session , hand -on project ,   camaraderi   fellow engin student .  day , luca stumbl   old , abandon workshop   outskirt  town .  sight  forget tool  discard materi spark  fire   . inspir   potenti   neglect space ,  decid  breath new life    creat  collabor hub  engin student . luca ralli  group  like -mind friend ,    transform  workshop     " innov forg . "    haven  student  work  person project , exchang idea ,  bring  engin dream  life .  space buzz   sound  drill ,  clank  metal ,   hum  3d printer  student collabor    robot  sustain energi solut .   innov forg gain popular , luca   team decid  organ  annual engin expo , showcas  innov project  emerg   workshop .  expo   attract student  faculti   local busi  industri profession keen  discov emerg talent .  year , luca   team develop  prototyp   solar -power irrig system design  help local farmer  summitvil .  project gain attent     engin prowess     potenti    real impact   commun .  innov forg   focal point  collabor project  aim  address real -world challeng . luca  journey  colleg     earn  degre ;    leav  last mark   engin commun .  graduat approach ,  innov forg continu  thrive ,  luca feel  sens  pride   legaci   creat .  town  summitvil    vibrant hub  innov , thank   hard work  dedic  luca turner .  colleg experi   testament   transform power  hand -on learn , collabor ,   drive    posit impact   world  engin ingenu .
""",
"""
Once upon a time in the bustling city of Arcadia, there lived a spirited and ambitious young woman named Emma Turner. Emma was a sophomore at Arcadia University, majoring in environmental science with a deep passion for sustainability and conservation. Her days were filled with classes, environmental club meetings, and the occasional part-time job at a local bookstore. One sunny afternoon, Emma received an invitation to participate in an environmental research project that would take her deep into the heart of the nearby Redwood Forest. The project, led by renowned ecologist Dr. Victoria Hayes, aimed to study the flora and fauna of the forest to better understand its ecosystem and potential threats. Excitement and anticipation bubbled within Emma as she packed her bags and set off on the adventure of a lifetime. Alongside a diverse group of fellow students, Emma embarked on a journey into the towering trees and lush undergrowth of the Redwood Forest. As the days unfolded, Emma found herself enchanted by the beauty and complexity of the ecosystem. She meticulously documented the various plant species, observed elusive wildlife, and collaborated with her peers on data collection. Amidst the research, Emma formed deep connections with her teammates, each of them bringing a unique perspective to the project. However, the tranquility of the forest was disrupted when the team stumbled upon signs of illegal logging. Deeply troubled by the discovery, Emma and her friends decided to take action. They used their findings to raise awareness both on campus and within the local community, organizing protests and initiating a campaign to protect the Redwood Forest. Emma's dedication caught the attention of local environmental organizations, and soon she found herself at the forefront of a movement to preserve not only the Redwood Forest but also other endangered ecosystems. Her passion and leadership skills shone brightly as she advocated for sustainable practices and conservation efforts. Through it all, Emma continued to balance her academic responsibilities, never losing sight of her dream to make a positive impact on the environment. As the campaign gained momentum, Emma's story became an inspiration to fellow students and activists alike. The once-unassuming college student had transformed into a force for change, proving that passion, determination, and a strong sense of purpose could make a significant difference in the world. And so, as the sun set over the Redwood Forest, casting a warm glow on the towering trees, Emma reflected on her journey. The adventure had not only shaped her college experience but had set her on a path to contribute meaningfully to the planet's well-being. In the heart of nature, surrounded by friends and a shared sense of purpose, Emma's story continued to unfold, leaving an indelible mark on both her and the world she sought to protect.
""",
"""
time   bustl citi  arcadia ,  live  spirit  ambiti young  name   .    sophomor  arcadia univers , major  environment scienc   deep passion  sustain  conserv .  day  fill  class , environment club meet ,   occasion  -time job   local bookstor .  sunni afternoon ,  receiv  invit  particip   environment research project     deep   heart   nearbi redwood forest .  project , lead  renown ecologist dr .   , aim  studi  flora  fauna   forest  well understand  ecosystem  potenti threat . excit  anticip bubbl     pack  bag  set    adventur   lifetim . alongsid  divers group  fellow student ,  embark   journey   tower tree  lush undergrowth   redwood forest .   day unfold ,  find  enchant   beauti  complex   ecosystem .  meticul document   plant speci , observ elus wildlif ,  collabor   peer  datum collect . amidst  research ,  form deep connect   teammat ,    bring  uniqu perspect   project .  ,  tranquil   forest  disrupt   team stumbl  sign  illeg log . deepli troubl   discoveri ,    friend decid   action .    find  rais awar   campu    local commun , organ protest  initi  campaign  protect  redwood forest .   dedic catch  attent  local environment organ ,  soon  find    forefront   movement  preserv    redwood forest    endang ecosystem .  passion  leadership skill shine brightli   advoc  sustain practic  conserv effort .    ,  continu  balanc  academ respons ,  lose sight   dream    posit impact   environ .   campaign gain momentum ,   stori   inspir  fellow student  activist alik .   -unassum colleg student  transform   forc  chang , prove  passion , determin ,   strong sens  purpos    signific differ   world .   ,   sun set   redwood forest , cast  warm glow   tower tree ,  reflect   journey .  adventur    shape  colleg experi   set    path  contribut meaning   planet   -be .   heart  natur , surround  friend   share sens  purpos ,   stori continu  unfold , leav  indel mark      world  seek  protect .
"""
]
transformedText = np.array([text_to_vectors(t) for t in text])
explainer = LimeTextExplainer(class_names=['female', 'male'])
predProb = model.predict_proba(transformedText)
pred = model.predict(transformedText)
predict_function = lambda x: model.predict_proba(np.array([text_to_vectors(x)]))
print("PRED", pred)
print("PRED PROB", predProb)

# Test with a single string
single_text = text[0]
print("Type of single_text:", type(single_text))
print("Single text:", single_text)
explanation = explainer.explain_instance(single_text, predict_function, num_features=20)
top_words_lime = explanation.as_list()
print("Top words for single text:")
print(top_words_lime)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
    
    
    # explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    # top_words_lime = explanation.as_list()
    # print(f"Top words for text response {i + 1}:")
    # masculineWords = []
    # feminineWords = []
    # for word, score in top_words_lime:
    #     if score > 0:
    #         masculineWords.append((word, score))
    #     else:
    #         feminineWords.append((word, score))
    # print("Masculine words: ", masculineWords)
    # print("Feminine words: ", feminineWords)
    # print("")

PRED ['female' 'male' 'male' 'male' 'female' 'female' 'male' 'male' 'female'
 'female' 'male' 'female' 'male' 'female']
PRED PROB [[0.51 0.49]
 [0.45 0.55]
 [0.48 0.52]
 [0.46 0.54]
 [0.5  0.5 ]
 [0.53 0.47]
 [0.46 0.54]
 [0.43 0.57]
 [0.5  0.5 ]
 [0.52 0.48]
 [0.46 0.54]
 [0.57 0.43]
 [0.49 0.51]
 [0.58 0.42]]
Type of single_text: <class 'str'>
Single text: 
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hand

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>

Word Embeddings using no ML model and only cosine similarity with "man" and "woman"

In [ ]:
# load word vectors
wordVectors = KeyedVectors.load_word2vec_format('../wordEmbeddings/GoogleNews-vectors-negative300.bin', binary=True)
manVector = wordVectors['man']
womanVector = wordVectors['woman']

# get average vector
def text_to_average_vector(text):
    if text.strip() == "":
        return None
    words = text.split()
    vectors = [wordVectors[word] for word in words if word in wordVectors.vocab]
    if len(vectors) == 0:
        return None
    return np.mean(vectors, axis=0)

# use cosine similarity to predict bais
def predict_bias(inputText):
    similaritiesToMan = []
    similaritiesToWoman = []
    words = inputText.split()
    
    # go through each word and calculate cosine similarity
    for word in words:
        if word in wordVectors.key_to_index:
            similarityToMan = cosine_similarity(wordVectors[word].reshape(1, -1), manVector.reshape(1, -1))
            similarityToWoman = cosine_similarity(wordVectors[word].reshape(1, -1), womanVector.reshape(1, -1))
            similaritiesToMan.append(similarityToMan)
            similaritiesToWoman.append(similarityToWoman)
    
    # calculate average
    if len(similaritiesToMan) == 0 or len(similaritiesToWoman) == 0:
        return None, None
    avgSimilarityToMan = np.mean(similaritiesToMan)
    avgSimilarityToWoman = np.mean(similaritiesToWoman)
    
    # normalize
    probMaleBias = (avgSimilarityToMan / (avgSimilarityToMan + avgSimilarityToWoman)) * 100
    probFemaleBias = (avgSimilarityToWoman / (avgSimilarityToMan + avgSimilarityToWoman)) * 100
    
    return probMaleBias, probFemaleBias

# testing
inputText = """
Once upon a time, in the bustling city of Academia, there lived an ambitious scientist named Alex. He was a dedicated soul with a passion to unravel the mysteries of science. Alex led an insatiable curiosity that led him to the hallowed halls of the prestigious Kepler University. From a humble background, Alex worked tirelessly to secure a scholarship that would pave the way to his academic journey. The moment he stepped onto the university grounds, he felt a surge of determination to make the most of the golden opportunity. Alex's days were filled with challenging lectures, fascinating experiments, countless hours in the library. His favorite haunt was the physics lab, where he immersed himself in the world of particles, equations, the boundless wonders of the universe. The spark in his eyes ignited with each breakthrough, no matter the size. Outside the realm of academia, Alex was known to be kind to lend a helping hand. Tutoring a struggling classmate or participating in community service projects, he believed in the power of knowledge to uplift not just himself but those around him. As the semesters passed, Alex's reputation as a brilliant, compassionate scientist grew. Professors admired his commitment to learning; fellow scientists sought his guidance. Alex remained humble, always acknowledging the contributions of others, fostering a sense of camaraderie.
"""
addSpaceBeforeText = add_space_before(inputText)
inputText = removeUnnecessaryWords(addSpaceBeforeText)

probMaleBias, probFemaleBias = predict_bias(inputText)

print("Male Percentage:", probMaleBias)
print("Female Percentage:", probFemaleBias)

if probMaleBias > probFemaleBias:
    bias = "male" 
else:
    bias = "female"
print("Predicted bias:", bias)

# bias scores for each word
biasScores = {}
words = inputText.split()
for word in words:
    if word in wordVectors.key_to_index:
        similarityToMan = cosine_similarity(wordVectors[word].reshape(1, -1), manVector.reshape(1, -1))
        similarityToWoman = cosine_similarity(wordVectors[word].reshape(1, -1), womanVector.reshape(1, -1))
        biasScores[word] = (similarityToMan - similarityToWoman)[0][0]
sortedBiasScores = sorted(biasScores.items(), key=lambda item: item[1], reverse=True)

masculineWords = [(word, score) for word, score in sortedBiasScores if score > 0][:10]
feminineWords = [(word, score) for word, score in sortedBiasScores if score < 0][:10]

print("Masculine words:", masculineWords)
print("Feminine words:", feminineWords)


Male Percentage: 56.885313987731934
Female Percentage: 43.11468303203583
Predicted bias: male
Masculine words: [('brilliant', 0.11809299), ('favorit', 0.11309473), ('experi', 0.08384444), ('commit', 0.07793382), ('physic', 0.07535678), ('fellow', 0.072152555), ('boundless', 0.070789225), ('pass', 0.06676336), ('hallow', 0.062647), ('univers', 0.058935393)]
Feminine words: [('realm', -0.0017680153), ('opportun', -0.005584955), ('lend', -0.006147187), ('hour', -0.010244541), ('background', -0.010786898), ('curios', -0.015917696), ('compassion', -0.016747624), ('feel', -0.01801464), ('journey', -0.018164247), ('hall', -0.01831232)]
